<a href="https://colab.research.google.com/github/davidcechak/EmbedDTI/blob/master/EmbedDTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/davidcechak/EmbedDTI.git

fatal: destination path 'EmbedDTI' already exists and is not an empty directory.


In [2]:
!ls

data  EmbedDTI	sample_data


In [3]:
!cd EmbedDTI

In [4]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# FIX torch-sparse bug https://stackoverflow.com/a/71764992
import torch 
print(torch.__version__)
!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster --yes
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

1.12.0+cu113
Found existing installation: torch-scatter 2.0.9
Uninstalling torch-scatter-2.0.9:
  Successfully uninstalled torch-scatter-2.0.9
Found existing installation: torch-sparse 0.6.14
Uninstalling torch-sparse-0.6.14:
  Successfully uninstalled torch-sparse-0.6.14
Found existing installation: torch-geometric 2.0.5
Uninstalling torch-geometric-2.0.5:
  Successfully uninstalled torch-geometric-2.0.5
Found existing installation: torch-cluster 1.6.0
Uninstalling torch-cluster-1.6.0:
  Successfully uninstalled torch-cluster-1.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
  Using cached https://data.pyg.org/whl/torch-1.12.0%2Bcu113/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl (7.9 MB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
  Using cac

# Functions for drug processing

In [7]:
#coding=utf-8
import pandas as pd
import numpy as np
import os
import json,pickle
from collections import OrderedDict
from rdkit import Chem
from rdkit.Chem import MolFromSmiles
import networkx as nx
from scipy.sparse import csr_matrix
# imports TestbedDataset, rmse, mse, pearson, spearman, ci
from EmbedDTI.utils import *
from collections import defaultdict
from rdkit.Chem.EnumerateStereoisomers import EnumerateStereoisomers, StereoEnumerationOptions
from rdkit.Chem import AllChem


# 处理SMILES序列，转化为原子的图结构
# Process SMILES sequences into graph structures of atoms
def atom_features(atom):
    return np.array(one_of_k_encoding_unk(atom.GetSymbol(),['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na','Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb','Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H','Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr','Cr', 'Pt', 'Hg', 'Pb', 'Unknown']) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) + # 返回原子上隐式Hs的数量
                    one_of_k_encoding_unk(atom.GetTotalValence(),[0,1,2,3,4,5,6,7,8,9,10]) +
                    one_of_k_encoding_unk(atom.GetFormalCharge(),[0,1,2,3,4,5,6,7,8,9,10]) +
                    [atom.GetIsAromatic()] +
                    [atom.IsInRing()]
                    )

def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))

def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))


def smile_to_graph(smile):
    mol = Chem.MolFromSmiles(smile)
    
    c_size = mol.GetNumAtoms()  # 原子数 Atomic number
    
    features = [] 
    for atom in mol.GetAtoms():
        feature = atom_features(atom)
        features.append( feature / sum(feature) )

    edges = []
    for bond in mol.GetBonds():
        edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
    # 构建有向图 Building a directed graph
    g = nx.Graph(edges).to_directed()
    edge_index = []
    for e1, e2 in g.edges:
        edge_index.append([e1, e2])
    return c_size, features, edge_index


# # 处理SMILES序列，转化为基团的图结构 Process SMILES sequences into graph structures of groups

MST_MAX_WEIGHT = 100 

def get_mol(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: 
        return None
    Chem.Kekulize(mol)
    return mol

# 构建邻接矩阵和特征矩阵
# Build Adjacency Matrix and Feature Matrix
# the "idx" argument is not used in the cluster_graph function
# originally: def cluster_graph(mol,idx):
def cluster_graph(mol,idx=None):
	n_atoms = mol.GetNumAtoms()
	# if n_atoms == 1: #special case
 #    	return [[0]], []
	cliques = []
	for bond in mol.GetBonds():
		a1 = bond.GetBeginAtom().GetIdx()
		a2 = bond.GetEndAtom().GetIdx()
		if not bond.IsInRing():
			cliques.append([a1,a2])

	ssr = [list(x) for x in Chem.GetSymmSSSR(mol)]
	cliques.extend(ssr)

	# nei_list为原子属于哪个基团
	nei_list = [[] for i in range(n_atoms)]
	for i in range(len(cliques)):
	    for atom in cliques[i]:
	        nei_list[atom].append(i)

	#Merge Rings with intersection > 2 atoms
	for i in range(len(cliques)):
		if len(cliques[i]) <= 2: continue
		for atom in cliques[i]:
			for j in nei_list[atom]:
				if i >= j or len(cliques[j]) <= 2: 
					continue
				inter = set(cliques[i]) & set(cliques[j])
				if len(inter) > 2:
					cliques[i].extend(cliques[j])
					cliques[i] = list(set(cliques[i]))
					cliques[j] = []
    
	cliques = [c for c in cliques if len(c) > 0]
	nei_list = [[] for i in range(n_atoms)]
	for i in range(len(cliques)):
		for atom in cliques[i]:
			nei_list[atom].append(i)

	# Build edges and add singleton cliques
	edges = defaultdict(int)
	for atom in range(n_atoms):
		if len(nei_list[atom]) <= 1: 
			continue
		cnei = nei_list[atom]
		bonds = [c for c in cnei if len(cliques[c]) == 2]
		rings = [c for c in cnei if len(cliques[c]) > 4]
		
		for i in range(len(cnei)):
			for j in range(i + 1, len(cnei)):
				c1,c2 = cnei[i],cnei[j]
				inter = set(cliques[c1]) & set(cliques[c2])
				if edges[(c1,c2)] < len(inter):
					edges[(c1,c2)] = len(inter) # cnei[i] < cnei[j] by construction
					edges[(c2,c1)] = len(inter)

	edges = [u + (MST_MAX_WEIGHT-v,) for u,v in edges.items()]
	row,col,data = zip(*edges)
	data = list(data)
	for i in range(len(data)):
		data[i] = 1
	data = tuple(data)
	n_clique = len(cliques)
	clique_graph = csr_matrix( (data,(row,col)), shape=(n_clique,n_clique) )
	edges = [[row[i],col[i]] for i in range(len(row))]
	
	return cliques, edges



def clique_features(clique,edges,clique_idx,smile):
    NumAtoms = len(clique) # 基团中除去氢原子的原子数
    NumEdges = 0  # 与基团所连的边数
    for edge in edges:
        if clique_idx == edge[0] or clique_idx == edge[1]:
            NumEdges += 1
    mol = Chem.MolFromSmiles(smile)
    NumHs = 0 # 基团中氢原子的个数
    for atom in mol.GetAtoms():
        if atom.GetAtomMapNum() in clique:
            NumHs += atom.GetTotalNumHs()
    # 基团中是否包含环
    IsRing = 0
    if len(clique) > 2:
        IsRing = 1
    # 基团中是否有键
    IsBond = 0
    if len(clique) == 2:
        IsBond = 1

    return np.array(one_of_k_encoding_unk(NumAtoms,[0,1,2,3,4,5,6,7,8,9,10]) + 
        one_of_k_encoding_unk(NumEdges,[0,1,2,3,4,5,6,7,8,9,10]) + 
        one_of_k_encoding_unk(NumHs,[0,1,2,3,4,5,6,7,8,9,10]) + 
        [IsRing] + 
        [IsBond]
        )






# Call embedDTI_smile_preprocess(...) to get the atom graph and substructures graph

In [8]:
def embedDTI_smile_preprocess(smile_to_process, test_MolToSmiles):
    # atom representation
    if (test_MolToSmiles):
        # do we need to put SMILES into canonical representation or not?
        smile_to_process = Chem.MolToSmiles(Chem.MolFromSmiles(smile_to_process),isomericSmiles=True)
    atom_result = smile_to_graph(smile_to_process)

    # substructure representation
    our_mol = get_mol(smile_to_process)
    our_clique, our_edge = cluster_graph(our_mol)
    c_features = []
    for idx in range(len(our_clique)):
        # TODO check if it has to be canonical SMILES or not
        cq_features = clique_features(our_clique[idx],our_edge,idx,smile_to_process)
        c_features.append( cq_features / sum(cq_features) )
    clique_size = len(our_clique)
    graph = (clique_size, c_features, our_edge)

    return atom_result, graph


Get the data

In [9]:
def read_data(filename):
    df = pd.read_csv('EmbedDTI/data/'+filename)
    drugs,proteins,Y = list(df['compound_iso_smiles']),list(df['target_sequence']),list(df['affinity'])

    return drugs,proteins,Y


# read files(train and test)
davis_train_drugs,davis_train_proteins,davis_train_Y = read_data("davis_train.csv")
davis_test_drugs,davis_test_proteins,davis_test_Y = read_data("davis_test.csv")
drugs = davis_train_drugs+davis_test_drugs
print(len(drugs)) # 30056


30056


Explore data dimensions

In [10]:
# our_smile = 'CN(C)CCOC(C1=CC=CC=C1)C1=CC=CC=C1'
for i in [0,10,11,32,4,5]:
  our_smile = drugs[i]
  atom_result_A, graph_A = embedDTI_smile_preprocess(our_smile, True)
  # atom_result_B, graph_B = embedDTI_smile_preprocess(our_smile, False)
  
  print(type(atom_result_A))
  print(np.array(atom_result_A[0]))
  print(np.array(atom_result_A[1]).shape)
  print(np.array(atom_result_A[2]).shape)
  print('__________________')

  print(type(graph_A))
  print(np.array(graph_A[0]))
  print(np.array(graph_A[1]).shape)
  print(np.array(graph_A[2]).shape)
  print('__________________')



<class 'tuple'>
25
(25, 101)
(54, 2)
__________________
<class 'tuple'>
13
(13, 35)
(28, 2)
__________________
<class 'tuple'>
19
(19, 101)
(42, 2)
__________________
<class 'tuple'>
8
(8, 35)
(16, 2)
__________________
<class 'tuple'>
35
(35, 101)
(80, 2)
__________________
<class 'tuple'>
14
(14, 35)
(32, 2)
__________________
<class 'tuple'>
33
(33, 101)
(74, 2)
__________________
<class 'tuple'>
16
(16, 35)
(30, 2)
__________________
<class 'tuple'>
34
(34, 101)
(74, 2)
__________________
<class 'tuple'>
19
(19, 35)
(40, 2)
__________________
<class 'tuple'>
34
(34, 101)
(74, 2)
__________________
<class 'tuple'>
19
(19, 35)
(40, 2)
__________________
